<a href="https://colab.research.google.com/github/SibongileT/plot_generator/blob/main/GetMovies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fake-useragent

In [1]:
from bs4 import BeautifulSoup
import json
import urllib
from time import sleep
import pandas as pd
import pickle
import requests
from random import randint
from time import sleep
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CSV created. Do not run code below again unless change need to be made

In [ ]:


def parse(url):
    '''
    Get scrape a page for it's title, titleid, and genre
    '''
    df = pd.DataFrame()
    movie_list = []
    id_list = []
    genre_list = []
    response = requests.get(url)
    print(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    bs = BeautifulSoup(page, 'lxml')
    movies = bs.find_all('div', attrs={'class':'lister-item-content'})
    for movie in movies:
        title = movie.find('a').get_text()
        href = movie.find('a').attrs['href']
        synopsis = movie.find_all('p')[1]
        link = synopsis.find('a')
        genre = movie.find('span', attrs={'class':'genre'})
        if genre:
            genre = genre.get_text().strip()
        else:
            genre = ''
        if href[7:17][9] == '/':
            title_id = href[7:16]
        else:
            title_id = href[7:17]
        movie_list.append(title)
        id_list.append(title_id)
        genre_list.append(genre)
    sleep(randint(2,10))
    return movie_list,id_list,genre_list

movie_info_df = pd.DataFrame(columns=['title','title_id','genres'])
movies_list = []
ids_list = []
genres_list = []
temp_df = pd.DataFrame(columns=['title','title_id','genre_list'])
for i in range(1,7):
        movie_list,id_list,genre_list = parse('https://www.imdb.com/list/ls057163321/?sort=release_date,desc&st_dt=&mode=detail&page='+str(i))
        movies_list.extend(movie_list)
        print(movies_list)
        ids_list.extend(id_list)
        print(ids_list)
        genres_list.append(genre_list)
        print(genres_list)
        temp_df = pd.DataFrame(columns=['title','title_id','genres'])
        temp_df['title'] = movies_list
        temp_df['title_id'] = ids_list
        temp_df['genres'] = genres_list[0]
        movies_list=[]
        ids_list=[]
        genres_list = []
        movie_info_df = movie_info_df.append(temp_df,ignore_index = True)
        movie_info_df.to_csv("movie_info.csv")
        !cp movie_info.csv "drive/My Drive/OPGFiles"
        sleep(randint(2,10)) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
https://www.imdb.com/list/ls057163321/?sort=release_date,desc&st_dt=&mode=detail&page=1
['The Post', 'Phantom Thread', 'Three Billboards Outside Ebbing, Missouri', 'Lady Bird', 'Darkest Hour', 'The Shape of Water', 'Dunkirk', 'Get Out', 'Call Me by Your Name', 'Fences', 'Hidden Figures', 'Lion', 'Hacksaw Ridge', 'Moonlight', 'Arrival', 'La La Land', 'Hell or High Water', 'Manchester by the Sea', 'The Revenant', 'The Big Short', 'Bridge of Spies', 'The Martian', 'Room', 'Spotlight', 'Mad Max: Fury Road', 'Brooklyn', 'American Sniper', 'Selma', 'The Theory of Everything', 'The Imitation Game', 'Birdman or (The Unexpected Virtue of Ignorance)', 'The Grand Budapest Hotel', 'Boyhood', 'Whiplash', 'American Hustle', 'The Wolf of Wall Street', 'Her', 'Captain Phillips', 'Dallas Buyers Club', 'Philomena', '12 Years a Slave', 'Gravity', 'Nebraska', 'Django Unchained',

In [2]:
movie_info_df = pd.read_csv('/content/drive/MyDrive/OPGFiles/movie_info.csv')

In [3]:
movie_info_2019_df = pd.read_csv('/content/drive/MyDrive/OPGFiles/Movie_Info_2019_2020.csv')

In [4]:
movie_info_df = movie_info_df.append(movie_info_2019_df)

In [5]:
movie_info_df.reset_index(inplace=True,drop=True)

In [12]:
!pip install imdbpy

     |████████████████████████████████| 307kB 7.7MB/s 


In [13]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()

# get a movie
movie = ia.get_movie('1392190')

# print the names of the directors of the movie
print('Directors:')
for director in movie['directors']:
    print(director['name'])

# print the genres of the movie
print('Genres:')
for genre in movie['genres']:
    print(genre)

movie['plot']

raw_release_date = ia.get_movie_release_dates('1392190')
date = raw_release_date['data']['raw release dates'][0]['date']

date

Directors:
George Miller
Genres:
Action
Adventure
Sci-Fi
Thriller


'7 May 2015'

In [ ]:
len(movie_info_df)

557

In [ ]:
movie['plot']

['When a beautiful stranger leads computer hacker Neo to a forbidding underworld, he discovers the shocking truth--the life he knows is the elaborate deception of an evil cyber-intelligence.',
 'Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. As a rebel against the machines, Neo must confront the agents: super-powerful computer programs devoted to stopping Neo and the entire human rebellion.::redcommander27',
 "Have you ever had a dream that you were so sure was real? What if you couldn't awaken? How would you know the difference between dream and reality? When a beautiful stranger (Carrie-Anne Moss) leads computer hacker Neo (Keanu Reeves) to a forbidding underworld, he discovers the sh

In [14]:
from random import randint
from time import sleep
import pandas as pd


def get_plots(ids):
    date_list = []
    id_list = []
    plot_list=[]
    director_list = []
    directors_list = []
    
    count=0
    plot_df = pd.DataFrame(columns=['title_id','release_date','directors','plot'])
    for col in ids:
        count+=1
        print(count)
        try:
            movie = ia.get_movie(col[2:])
            raw_release_date = ia.get_movie_release_dates(col[2:])
            try:
                date = raw_release_date['data']['raw release dates'][0]['date']
            except:
                date = None
            date_list.append(date)
            for director in movie['directors']:
                   director_list.append(director['name'])
            directors_list.append(director_list)
            #Empty the directors list so that the director for the next movie can be added
            director_list=[]
            
            #Get plot and add it to list so i can be appended to the dataframe
            plot= movie['plot']
            plot_list.append(plot)
            id_list.append(col)
            
            if count%557 == 0:
                ## Create a df containing plots
                df = pd.DataFrame(columns=['title_id','release_date','directors','plot'])
                df['plot'] = plot_list
                df['release_date'] = date_list
                df['title_id']=id_list
                df['director'] = directors_list
        

                ##After adding to temp dataframe empty list
                date_list = []
                id_list=[]
                plot_list = [] 
                directors_list = []

                ##Append and pickle plot
                plot_df = plot_df.append(df,ignore_index = True)
                plot_df.to_csv("cleaned_plots.csv")
                !cp cleaned_plots.csv "drive/My Drive/OPGFiles"
                sleep(randint(2,10))
        except:
                print(col)
                pass
    return date_list,plot_list,id_list,director_list

In [15]:
date_list,plot_list,id_list,director_list = get_plots(movie_info_df['title_id'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
movie_plot_df = pd.read_csv('/content/drive/MyDrive/OPGFiles/cleaned_plots.csv',index_col=0)

In [19]:
movie_plot_df = movie_plot_df.drop(columns=['directors'])